사람 + 총 + 칼 인식
pose estimation은 모든 사람에 대해 인식(수정해서 아래 경우에만 적용할 수도 있음)
사람과 총 혹은 칼의 bounding box가 겹치는 경우에만 arcface 적용

In [ ]:
!pip install opencv-python


In [1]:
#1. 얼굴 학습시키는 부분
import cv2
import numpy as np
import os
from insightface.app import FaceAnalysis

def initialize_arcface():
    app = FaceAnalysis(name="buffalo_l")  # ArcFace 모델 (buffalo_l은 기본 권장)
    app.prepare(ctx_id=-1, det_size=(640, 640))  # GPU: ctx_id=0, CPU: -1
    return app

def get_face_embedding(app, image_bgr):
    # ArcFace의 app.get()은 BGR 형식으로 이미지를 받기도 합니다.
    # 만약 RGB가 필요하면 cvtColor로 변환하세요.
    faces = app.get(image_bgr)
    if len(faces) > 0:
        return faces[0].embedding  # 첫 번째 얼굴의 임베딩
    else:
        return None

def generate_average_embedding(app, folder_path):
    embeddings = []
    for file in os.listdir(folder_path):
        if file.lower().endswith(('.jpg', '.jpeg', '.png')):
            img_path = os.path.join(folder_path, file)
            image = cv2.imread(img_path)
            if image is None:
                print(f"이미지 로드 실패: {img_path}")
                continue
            
            embedding = get_face_embedding(app, image)
            if embedding is not None:
                embeddings.append(embedding)
            else:
                print(f"얼굴 검출 실패: {img_path}")
    
    if len(embeddings) == 0:
        raise ValueError("임베딩을 하나도 생성하지 못했습니다.")
    
    avg_embedding = np.mean(embeddings, axis=0)
    return avg_embedding

if __name__ == "__main__":
    app = initialize_arcface()
    # 내 얼굴 사진 폴더
    my_face_folder = "C:/myface"  
    my_face_embedding = generate_average_embedding(app, my_face_folder)
    np.save("my_face_embedding.npy", my_face_embedding)  # 필요 시 저장
    print("내 얼굴 평균 임베딩 생성 완료.")

ModuleNotFoundError: No module named 'cv2'

In [ ]:
import cv2
import numpy as np
import rospy
from std_msgs.msg import Bool
from ultralytics import YOLO
from insightface.app import FaceAnalysis
from sklearn.metrics.pairwise import cosine_similarity
import torch
import mediapipe as mp

# ================================
# ROS 초기화
# ================================
rospy.init_node('pose_emergency_detector', anonymous=True)
emergency_pub = rospy.Publisher('/is_emergency', Bool, queue_size=1)

# ================================
# 모델 초기화
# ================================
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# YOLOv8 불러오기
model = YOLO("C:/epoch180.pt").to(device)

# ArcFace 설정
arc_app = FaceAnalysis(name='buffalo_l')
arc_app.prepare(ctx_id=0 if torch.cuda.is_available() else -1)

# MediaPipe Pose
mp_pose = mp.solutions.pose
pose_model = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_draw = mp.solutions.drawing_utils

# ================================
# 카메라 열기
# ================================
cap = cv2.VideoCapture(0)

def is_arm_raised(landmarks):
    """팔이 어깨보다 위로 올라가면 True"""
    left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]
    right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]
    left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
    right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]

    left_up = left_wrist.y < left_shoulder.y
    right_up = right_wrist.y < right_shoulder.y
    return left_up or right_up

while not rospy.is_shutdown():
    ret, frame = cap.read()
    if not ret:
        continue

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # YOLO 감지
    results = model(frame)[0]

    # 위험 판별 변수 초기화
    is_emergency = False

    # 얼굴 추출
    faces = arc_app.get(rgb_frame)
    embeddings = [f['embedding'] for f in faces]

    # 포즈 추정
    pose_results = pose_model.process(rgb_frame)
    if pose_results.pose_landmarks:
        mp_draw.draw_landmarks(frame, pose_results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        landmarks = pose_results.pose_landmarks.landmark
        if is_arm_raised(landmarks):
            is_emergency = True

    # ROS publish
    emergency_pub.publish(Bool(data=is_emergency))

    # 디버깅용 표시
    if is_emergency:
        cv2.putText(frame, "EMERGENCY", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,0,255), 3)

    cv2.imshow("Pose + Face", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
